In [35]:
import cv2
import os
import random
import numpy as np
from glob import glob
from tqdm import tqdm

def extract_tool(img, img_path="unknown"):
    """
    Вырезает инструмент из изображения без лишнего фона.
    Возвращает tool_img и маску tool_mask.
    """
    if img is None:
        print(f"Ошибка: изображение не загружено: {img_path}")
        return None, None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Попробуем несколько методов пороговой обработки
    # 1. Комбинация Otsu и адаптивного порога
    _, otsu_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    adapt_mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY_INV, 21, 5)  # Увеличен blockSize
    mask = cv2.bitwise_and(otsu_mask, adapt_mask)

    # 2. Если маска пуста, пробуем Canny для выделения краёв
    if not np.any(mask):
        print(f"Пробуем Canny для {img_path}")
        edges = cv2.Canny(gray, 50, 150)
        mask = cv2.dilate(edges, np.ones((5, 5), np.uint8), iterations=2)

    # Убираем шум
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Находим контуры
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        print(f"Контуры не найдены для изображения: {img_path}")
        # Сохраняем проблемное изображение и маску для отладки
        cv2.imwrite(f"debug_failed_img_{os.path.basename(img_path)}.jpg", img)
        cv2.imwrite(f"debug_failed_mask_{os.path.basename(img_path)}.jpg", mask)
        return None, None

    # Объединяем контуры с площадью > 50 пикселей (уменьшено для большей чувствительности)
    full_mask = np.zeros_like(mask)
    valid_contours = [c for c in contours if cv2.contourArea(c) > 50]
    if not valid_contours:
        print(f"Нет контуров с площадью > 50 для {img_path}")
        cv2.imwrite(f"debug_failed_img_{os.path.basename(img_path)}.jpg", img)
        cv2.imwrite(f"debug_failed_mask_{os.path.basename(img_path)}.jpg", mask)
        return None, None
    cv2.drawContours(full_mask, valid_contours, -1, 255, thickness=cv2.FILLED)

    # Bounding box по маске
    x, y, w, h = cv2.boundingRect(full_mask)
    tool_img = img[y:y+h, x:x+w]
    tool_mask = full_mask[y:y+h, x:x+w]

    # Сохраняем маску для отладки (случайно, чтобы не перегружать диск)
    if random.random() < 0.1:
        cv2.imwrite(f"debug_mask_{os.path.basename(img_path)}_{random.randint(0, 1000)}.jpg", tool_mask)

    return tool_img, tool_mask


def overlay_with_mask(base_img, overlay_img, cls_id, boxes, img_path="unknown"):
    """
    Накладывает overlay_img на base_img строго по инструменту.
    """
    tool_img, tool_mask = extract_tool(overlay_img, img_path)
    if tool_img is None:
        print(f"Не удалось извлечь инструмент из {img_path}")
        return base_img, boxes

    # Масштабирование инструмента
    h, w = tool_img.shape[:2]
    scale = random.uniform(0.5, 1.5)  # Увеличен диапазон для видимости
    new_w, new_h = int(w * scale), int(h * scale)
    tool_img = cv2.resize(tool_img, (new_w, new_h))
    tool_mask = cv2.resize(tool_mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

    # Поворот инструмента
    angle = random.randint(0, 360)
    M = cv2.getRotationMatrix2D((new_w // 2, new_h // 2), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    bound_w = int(new_h * sin + new_w * cos)
    bound_h = int(new_h * cos + new_w * sin)
    M[0, 2] += (bound_w / 2) - new_w / 2
    M[1, 2] += (bound_h / 2) - new_h / 2
    tool_img = cv2.warpAffine(tool_img, M, (bound_w, bound_h), borderValue=(255, 255, 255))
    tool_mask = cv2.warpAffine(tool_mask, M, (bound_w, bound_h), borderValue=0)
    new_w, new_h = bound_w, bound_h

    # Случайная позиция на base_img
    max_y = max(0, base_img.shape[0] - new_h)
    max_x = max(0, base_img.shape[1] - new_w)
    y_offset = random.randint(0, max_y) if max_y > 0 else 0
    x_offset = random.randint(0, max_x) if max_x > 0 else 0

    # Проверяем границы
    h_roi = min(new_h, base_img.shape[0] - y_offset)
    w_roi = min(new_w, base_img.shape[1] - x_offset)
    tool_img = tool_img[:h_roi, :w_roi]
    tool_mask = tool_mask[:h_roi, :w_roi]

    roi = base_img[y_offset:y_offset + h_roi, x_offset:x_offset + w_roi]

    # Накладываем инструмент
    mask_inv = cv2.bitwise_not(tool_mask)
    bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
    fg = cv2.bitwise_and(tool_img, tool_img, mask=tool_mask)
    dst = cv2.add(bg, fg)
    base_img[y_offset:y_offset + h_roi, x_offset:x_offset + w_roi] = dst

    # Пересчёт bbox YOLO
    coords = cv2.findNonZero(tool_mask)
    if coords is not None:
        x0, y0, w_box, h_box = cv2.boundingRect(coords)
        x0 += x_offset
        y0 += y_offset
        x_center = (x0 + w_box / 2) / base_img.shape[1]
        y_center = (y0 + h_box / 2) / base_img.shape[0]
        width = w_box / base_img.shape[1]
        height = h_box / base_img.shape[0]
        # Проверка корректности бокса
        if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 < width <= 1 and 0 < height <= 1:
            boxes.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        else:
            print(f"Некорректный бокс для {img_path}: {cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return base_img, boxes


def make_augmented_image(image_paths, class_ids, save_img_path, save_lbl_path):
    """Создаёт одно новое фото с несколькими инструментами без лишнего фона."""
    canvas_size = 512  # Уменьшен для большей плотности инструментов
    base_img = np.ones((canvas_size, canvas_size, 3), np.uint8) * 255
    boxes = []

    # Гарантируем хотя бы один инструмент
    success = False
    attempts = 0
    max_attempts = 20  # Увеличено для большей вероятности успеха
    while not success and attempts < max_attempts:
        overlay_img_path = random.choice(image_paths)
        overlay_img = cv2.imread(overlay_img_path)
        if overlay_img is None:
            print(f"Не удалось загрузить изображение: {overlay_img_path}")
            attempts += 1
            continue
        cls_id = class_ids[overlay_img_path]
        base_img, boxes = overlay_with_mask(base_img, overlay_img, cls_id, boxes, overlay_img_path)
        if boxes:
            success = True
        attempts += 1

    if not success:
        print(f"Не удалось наложить ни один инструмент для {save_img_path} после {max_attempts} попыток")
        return

    # Накладываем дополнительные инструменты (2–4)
    n_overlays = random.randint(2, 4)
    overlay_paths = random.sample(image_paths, min(n_overlays, len(image_paths)))
    for path in overlay_paths:
        overlay_img = cv2.imread(path)
        if overlay_img is None:
            print(f"Не удалось загрузить изображение: {path}")
            continue
        cls_id = class_ids[path]
        base_img, boxes = overlay_with_mask(base_img, overlay_img, cls_id, boxes, path)

    if not boxes:
        print(f"Не удалось создать изображение с инструментами: {save_img_path}")
        return

    # Сохраняем
    cv2.imwrite(save_img_path, base_img)
    with open(save_lbl_path, "w") as f:
        f.write("/n".join(boxes))
    print(f"Сохранено изображение: {save_img_path}, боксы: {len(boxes)}")


def generate_augmented_dataset(root_dir, output_dir, n_images=1000):
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

    # Классы = подпапки
    class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    class_names.sort()
    print("Найдены классы:", class_names)

    # Собираем все пути картинок и их классы
    image_paths = []
    class_ids = {}
    for cls_id, cls_name in enumerate(class_names):
        img_dir = os.path.join(root_dir, cls_name, f"{cls_name}_resized")
        for img_path in glob(os.path.join(img_dir, "*.jpg")) + glob(os.path.join(img_dir, "*.png")):
            image_paths.append(img_path)
            class_ids[img_path] = cls_id

    print(f"Всего изображений найдено: {len(image_paths)}")

    # Генерируем новые картинки с прогрессбаром
    for i in tqdm(range(n_images), desc="Генерация датасета"):
        save_img_path = os.path.join(output_dir, "images", f"aug_{i:05d}.jpg")
        save_lbl_path = os.path.join(output_dir, "labels", f"aug_{i:05d}.txt")

        make_augmented_image(image_paths, class_ids, save_img_path, save_lbl_path)

    # Создаём dataset.yaml
    yaml_path = os.path.join(output_dir, "dataset.yaml")
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write(f"path: {os.path.abspath(output_dir)}/n")
        f.write("train: images/n")
        f.write("val: images/n")
        f.write("test: images/n/n")
        f.write(f"nc: {len(class_names)}/n")
        f.write("names:/n")
        for i, name in enumerate(class_names):
            f.write(f"  {i}: {name}/n")

    print("✅ Датасет с наложением создан!")

In [36]:
generate_augmented_dataset("S:/hacatons/tools/class_data_tools", "S:/hacatons/tools/Tools_yolov8-8/data2")

Найдены классы: ['Adjustable_wrench', 'Offset_Phillips_screwdriver', 'Oil_can_opener', 'Open_end_wrench', 'Pliers', 'Rotary_wheel', 'Safety_pliers', 'Shernica', 'Side_cutters', 'screwdriver_1', 'screwdriver_2']
Всего изображений найдено: 5010


Генерация датасета:   0%|          | 0/1000 [00:00<?, ?it/s]

Генерация датасета:   0%|          | 2/1000 [00:00<01:06, 15.08it/s]

Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2497.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2497.JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00000.jpg, боксы: 4
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1380.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1380.JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00001.jpg, боксы: 3
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\screwdriver_2\screwdriver_2_resized\DSCN1541.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\screwdriver_2\screwdriver_2_resized\DSCN1541.JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00002.jpg, бок

Генерация датасета:   0%|          | 4/1000 [00:00<01:02, 15.90it/s]

...

Генерация датасета:  16%|█▋        | 164/1000 [00:09<00:51, 16.09it/s]

Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2580.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2580.JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00162.jpg, боксы: 4
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00163.jpg, боксы: 4
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1185.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1185.JPG
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\Open_end_wrench\Open_end_wrench_resized\DSCN2344 (2).JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\Open_end_wrench\Open_end_wrench_resized\DSCN2344 (2).JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\au

Генерация датасета:  17%|█▋        | 168/1000 [00:10<00:49, 16.65it/s]

Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00166.jpg, боксы: 3
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2491.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\screwdriver_1\screwdriver_1_resized\DSCN2491.JPG
Сохранено изображение: S:/hacatons/tools/Tools_yolov8-8/data2\images\aug_00167.jpg, боксы: 4
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1223.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\Adjustable_wrench\Adjustable_wrench_resized\DSCN1223.JPG
Контуры не найдены для изображения: S:/hacatons/tools/class_data_tools\Offset_Phillips_screwdriver\Offset_Phillips_screwdriver_resized\DSCN1873.JPG
Не удалось извлечь инструмент из S:/hacatons/tools/class_data_tools\Offset_Phillips_screwdriver\Offset_Phillips_screwdriver_resized\DSCN1873.JPG
Контуры не найдены для изображе

KeyboardInterrupt: 

In [52]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

def load_image(image_path: str) -> np.ndarray:
    """Load an image using matplotlib."""
    return plt.imread(image_path)

def save_image(image: np.ndarray, save_path: str):
    """Save an image using matplotlib."""
    plt.imsave(save_path, image)

def load_yolo_annotation(anno_path: str, image_shape: Tuple[int, int], class_override: int = None) -> List[Tuple[int, int, int, int, int]]:
    """
    Load YOLO annotation, handling malformed files where boxes are separated by '/n' instead of newlines.
    Returns list of (class_id, x_min, y_min, x_max, y_max) in absolute pixels.
    If class_override is provided, replaces all class_ids with it (for per-class datasets).
    """
    height, width = image_shape[:2]
    boxes = []
    with open(anno_path, 'r') as f:
        content = f.read().strip()
    # Split on '/n' to handle malformed separators
    lines = content.split('/n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) != 5:
            continue  # Skip invalid lines
        try:
            class_id = int(parts[0])
            if class_override is not None:
                class_id = class_override
            center_x = float(parts[1]) * width
            center_y = float(parts[2]) * height
            w = float(parts[3]) * width
            h = float(parts[4]) * height
            x_min = int(center_x - w / 2)
            y_min = int(center_y - h / 2)
            x_max = int(center_x + w / 2)
            y_max = int(center_y + h / 2)
            boxes.append((class_id, x_min, y_min, x_max, y_max))
        except ValueError:
            continue  # Skip if conversion fails
    return boxes

def save_yolo_annotation(anno_path: str, boxes: List[Tuple[int, int, int, int, int]], image_shape: Tuple[int, int]):
    """
    Save YOLO annotation.
    boxes: list of (class_id, x_min, y_min, x_max, y_max) in absolute pixels.
    """
    height, width = image_shape[:2]
    with open(anno_path, 'w') as f:
        for class_id, x_min, y_min, x_max, y_max in boxes:
            center_x = (x_min + x_max) / 2 / width
            center_y = (y_min + y_max) / 2 / height
            w = (x_max - x_min) / width
            h = (y_max - y_min) / height
            f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {w:.6f} {h:.6f}/n")

def extract_object(image: np.ndarray, box: Tuple[int, int, int, int, int]) -> Tuple[np.ndarray, np.ndarray]:
    """Extract object and create a mask (assuming simple crop, no alpha)."""
    _, x_min, y_min, x_max, y_max = box
    obj = image[y_min:y_max, x_min:x_max].copy()
    mask = np.ones_like(obj, dtype=bool)  # Simple mask, no transparency
    return obj, mask

def paste_object(base_image: np.ndarray, obj: np.ndarray, mask: np.ndarray, position: Tuple[int, int]) -> np.ndarray:
    """Paste object onto base image at position (top-left y, x)."""
    h, w = obj.shape[:2]
    y, x = position
    if y + h > base_image.shape[0] or x + w > base_image.shape[1]:
        return base_image  # Skip if out of bounds
    base_image[y:y+h, x:x+w][mask] = obj[mask]
    return base_image

def augment_dataset(root_dir: str, output_images_dir: str, output_labels_dir: str, num_augmentations: int = 10, overlap_factor: float = 0.5):
    """
    Augment YOLO dataset by overlaying objects from random images onto a base image,
    making objects appear stacked/on top of each other with some overlap.
    Collects data from class-specific subfolders in root_dir, similar to prepare_yolo_dataset.
    
    Assumes per-class labels have class_id 0, overrides with global class_idx.
    
    - root_dir: Root directory with class subfolders (e.g., class_data_tools).
    - output_images_dir: Directory to save new images.
    - output_labels_dir: Directory to save new annotations.
    - num_augmentations: Number of new images to generate.
    - overlap_factor: How much objects overlap (0-1, 1=full overlap).
    
    For each augmentation:
    - Pick a random base image from collected pairs.
    - Pick 1-3 random other images, extract their objects.
    - Paste objects onto base, stacking them with overlap (e.g., vertically shifted).
    - Update annotations for all objects (original + added), with class overrides.
    """
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    # Collect class_names and dataset_pairs like in prepare_yolo_dataset
    class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    class_names.sort()

    dataset_pairs = []  # (img_path, lbl_path, class_idx)
    missing_labels = 0

    for class_idx, class_name in enumerate(class_names):
        img_dir = os.path.join(root_dir, class_name, f"{class_name}_resized")
        lbl_dir = os.path.join(root_dir, class_name, f"{class_name}_NEW_labels_yolo")

        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            continue

        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith((".jpg", ".jpeg", ".png", ".JPG")):
                continue

            img_path = os.path.join(img_dir, img_file)
            lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")

            if not os.path.exists(lbl_path):
                missing_labels += 1
                continue

            dataset_pairs.append((img_path, lbl_path, class_idx))

    if not dataset_pairs:
        raise ValueError("No valid image-label pairs found in root_dir.")

    print(f"Found {len(dataset_pairs)} pairs, skipped {missing_labels} images without labels.")

    for i in range(num_augmentations):
        # Select base pair
        base_pair = random.choice(dataset_pairs)
        base_img_path, base_lbl_path, base_class_idx = base_pair
        base_image = load_image(base_img_path).copy()  # Writable copy

        if not os.path.exists(base_lbl_path):
            continue

        boxes = load_yolo_annotation(base_lbl_path, base_image.shape, class_override=base_class_idx)
        
        # Keep original boxes, add new ones
        all_boxes = boxes.copy()
        
        # Select 1-3 other pairs to overlay objects from
        num_overlays = random.randint(1, 3)
        other_pairs = random.sample([p for p in dataset_pairs if p != base_pair], min(num_overlays, len(dataset_pairs)-1))
        
        current_y_offset = 0  # For stacking vertically with overlap
        
        for other_pair in other_pairs:
            other_img_path, other_lbl_path, other_class_idx = other_pair
            other_image = load_image(other_img_path)
            
            if not os.path.exists(other_lbl_path):
                continue
            
            other_boxes = load_yolo_annotation(other_lbl_path, other_image.shape, class_override=other_class_idx)
            
            for box in other_boxes:
                obj, mask = extract_object(other_image, box)
                
                # Position: stack vertically with overlap
                overlap_pixels = int(obj.shape[0] * overlap_factor)
                position = (current_y_offset, random.randint(0, base_image.shape[1] - obj.shape[1]))
                
                base_image = paste_object(base_image, obj, mask, position)
                
                # Add new box to annotations
                class_id, _, _, _, _ = box  # Already overridden
                y_min = position[0]
                x_min = position[1]
                y_max = y_min + obj.shape[0]
                x_max = x_min + obj.shape[1]
                all_boxes.append((class_id, x_min, y_min, x_max, y_max))
                
                # Update offset for next
                current_y_offset += obj.shape[0] - overlap_pixels
        
        # Save new image and annotation
        base_file = os.path.basename(base_img_path)
        new_file = f"aug_{i}_{base_file}"
        new_path = os.path.join(output_images_dir, new_file)
        save_image(base_image, new_path)
        
        new_anno_file = os.path.splitext(new_file)[0] + '.txt'
        new_anno_path = os.path.join(output_labels_dir, new_anno_file)
        save_yolo_annotation(new_anno_path, all_boxes, base_image.shape)

# Example usage:
# augment_dataset('path/to/root_dir', 'path/to/output_images_dir', 'path/to/output_labels_dir', num_augmentations=10, overlap_factor=0.5)

In [54]:
augment_dataset('S:/hacatons/tools/class_data_tools', 'S:/hacatons/tools/Tools_yolov8-8/outImg', 'S:/hacatons/tools/Tools_yolov8-8/outLb', num_augmentations=1000, overlap_factor=0.5)

Found 4932 pairs, skipped 78 images without labels.


In [2]:
import os
import shutil
import random
from tqdm import tqdm

def prepare_flat_yolo_dataset(images_dir: str, labels_dir: str, output_dir: str, class_names: list[str], split=(0.8, 0.1, 0.1)):
    """
    Prepares a YOLO dataset from flat directories of images and labels.
    
    :param images_dir: Directory containing all image files (.jpg, .png, etc.).
    :param labels_dir: Directory containing all label files (.txt) with matching basenames to images.
    :param output_dir: Directory to save the YOLO-formatted dataset.
    :param class_names: List of class names corresponding to class IDs (0 to nc-1).
    :param split: Proportions for (train, val, test).
    """
    os.makedirs(output_dir, exist_ok=True)

    # Create YOLO folder structure
    for subset in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, "images", subset), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", subset), exist_ok=True)

    # Collect pairs
    dataset_pairs = []  # (img_path, lbl_path)
    missing_labels = 0

    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png", ".JPG"))]
    
    for img_file in image_files:
        img_path = os.path.join(images_dir, img_file)
        lbl_file = os.path.splitext(img_file)[0] + ".txt"
        lbl_path = os.path.join(labels_dir, lbl_file)

        if not os.path.exists(lbl_path):
            missing_labels += 1
            continue  # Skip images without labels

        dataset_pairs.append((img_path, lbl_path))

    print(f"Found {len(dataset_pairs)} image-label pairs")
    print(f"⚠️ Skipped {missing_labels} images without labels")

    # Shuffle and split
    random.shuffle(dataset_pairs)
    n = len(dataset_pairs)
    n_train = int(split[0] * n)
    n_val = int(split[1] * n)

    subsets = {
        "train": dataset_pairs[:n_train],
        "val": dataset_pairs[n_train:n_train + n_val],
        "test": dataset_pairs[n_train + n_val:]
    }

    for subset, pairs in subsets.items():
        for img_path, lbl_path in tqdm(pairs, desc=f"Copying {subset}", unit="file"):
            img_name = os.path.basename(img_path)
            lbl_name = os.path.basename(lbl_path)

            shutil.copy(img_path, os.path.join(output_dir, "images", subset, img_name))
            shutil.copy(lbl_path, os.path.join(output_dir, "labels", subset, lbl_name))

    # Create dataset.yaml
    yaml_path = os.path.join(output_dir, "dataset.yaml")
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write(f"path: {os.path.abspath(output_dir)}/n")
        f.write("train: images/train/n")
        f.write("val: images/val/n")
        f.write("test: images/test/n/n")
        f.write(f"nc: {len(class_names)}/n")
        f.write("names:/n")
        for i, name in enumerate(class_names):
            f.write(f"  {i}: {name}/n")

    print("✅ Dataset ready! dataset.yaml created.")

# Example usage:
# prepare_flat_yolo_dataset('path/to/images_dir', 'path/to/labels_dir', 'S:/hacatons/tools/Tools_yolov8-8/data', class_names)

In [3]:
class_names = [
    "Adjustable_wrench",
    "screwdriver_1",
    "screwdriver_2",
    "Offset_Phillips_screwdriver",
    "Side_cutters",
    "Shernica",
    "Safety_pliers",
    "Pliers",
    "Rotary_wheel",
    "Open_end_wrench",
    "Oil_can_opener"
]

In [4]:
prepare_flat_yolo_dataset('S:/hacatons/tools/Tools_yolov8-8/outImg', 'S:/hacatons/tools/Tools_yolov8-8/outLb', 'S:/hacatons/tools/Tools_yolov8-8/data2', class_names)


Found 1000 image-label pairs
⚠️ Skipped 0 images without labels


Copying test: 100%|██████████| 100/100 [00:00<00:00, 953.13file/s]

✅ Dataset ready! dataset.yaml created.


In [2]:
import albumentations as A
import cv2
import os
import shutil
from albumentations.pytorch import ToTensorV2

# Создаём пайплайн аугментаций
transform = A.Compose([
    # Геометрические трансформации
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.Rotate(limit=30, border_mode=cv2.BORDER_CONSTANT, p=0.5),
    A.RandomResizedCrop(
    height=640, width=640,
    scale=(0.8, 1.0),
    ratio=(0.75, 1.33),
    p=0.5
    )

    # Цветовые аугментации
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.MotionBlur(blur_limit=5, p=0.2),

    # Cutout / случайные квадраты
    A.CoarseDropout(
        max_holes=5, max_height=50, max_width=50,
        min_holes=1, min_height=20, min_width=20,
        fill_value=0, p=0.5
    ),

    # Дополнительно Mosaic / MixUp можно реализовать отдельно
], 
    bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['class_labels'],  # список классов
        min_visibility=0.2              # игнорируем боксы, если осталось <20%
    )
)

def read_yolo_labels(label_path):
    boxes, labels = [], []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            cls, x, y, w, h = line.strip().split()
            boxes.append([float(x), float(y), float(w), float(h)])
            labels.append(int(cls))
    return boxes, labels

def save_yolo_labels(label_path, boxes, labels):
    with open(label_path, 'w') as f:
        for box, cls in zip(boxes, labels):
            x, y, w, h = box
            f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def augment_dataset(images_dir, labels_dir, output_images_dir, output_labels_dir, num_aug=3):
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    for img_file in os.listdir(images_dir):
        if not (img_file.endswith(".jpg") or img_file.endswith(".png")):
            continue

        img_path = os.path.join(images_dir, img_file)
        label_file = img_file.rsplit(".", 1)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_file)

        if not os.path.exists(label_path):
            continue

        # Читаем изображение и лейблы
        image = cv2.imread(img_path)
        h, w = image.shape[:2]
        boxes, labels = read_yolo_labels(label_path)

        for i in range(num_aug):
            transformed = transform(image=image, bboxes=boxes, class_labels=labels)

            aug_img = transformed["image"]
            aug_boxes = transformed["bboxes"]
            aug_labels = transformed["class_labels"]

            # Сохраняем изображение
            aug_img_file = f"{img_file.rsplit('.', 1)[0]}_aug{i}.jpg"
            aug_img_path = os.path.join(output_images_dir, aug_img_file)
            cv2.imwrite(aug_img_path, aug_img)

            # Сохраняем аннотации
            aug_label_path = os.path.join(output_labels_dir, aug_img_file.rsplit('.', 1)[0] + ".txt")
            save_yolo_labels(aug_label_path, aug_boxes, aug_labels)



ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.8, 1.0), 'ra...: None, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
# Пример вызова
augment_dataset(
    "path/to/original/images",
    "path/to/original/labels",
    "path/to/aug/images",
    "path/to/aug/labels",
    num_aug=3
)